In [1]:
#manipulação de dados
import pandas as pd
import numpy as np

#automação
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#web scraping
import requests
from bs4 import BeautifulSoup

#string e data
import unidecode

#tempo
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)

In [2]:
browser = webdriver.Firefox()

In [3]:
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_UserName"]').send_keys('gabriel_s_anjos@yahoo.com') #e-mail
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_Password"]').send_keys('Gabriel2022!') #senha
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_btnLogin"]').click() #login

In [4]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [5]:
#click setor/profissional
browser.find_element("xpath", '//*[@id="s2id_autogen1"]/a/span[1]').click()
#click [profissional]
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()

# Capturando setores:

In [6]:
#click criar novo plantão
browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()

In [7]:
inicio = time.time() #iniciando o cronometro

#Setor
browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()

setor_html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

fim = time.time() #encerrando cronometro
print('\nTempo que demorou o codigo:', fim - inicio)


Tempo que demorou o codigo: 0.7650556564331055


In [8]:
inicio = time.time() #iniciando o cronometro

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1
            
        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

fim = time.time() #encerrando cronometro
print('\nTempo que demorou o codigo:', fim - inicio)




Tempo que demorou o codigo: 0.017661094665527344


In [ ]:
scrap_unidades.to_excel("setores.xlsx", index=False)
scrap_unidades.sample(20)

# TESTANDO SETORES

In [9]:
df = pd.read_excel('df_gabs.xlsx')
df.head()

,chat,dia,nome,mensagem,controle,data,unidade,extra,ENTRADA,SAIDA,DURAÇÃO,PERIODO,setor,sem_ou_fds
0,[27/04/2023 09:15:57] Rodrigo Gomes Minas Nova...,27/04/2023,RODRIGO GOMES MINAS NOVAS,27/04 Morumbi - Gomes entrada 06:15,NaN,27/04/2023,MORUMBI,NaN,06:00,07:00,01:00,6h,SEMANA - 6HS,semana
1,[27/04/2023 10:45:02] Jessica Lidia de Souza: ...,27/04/2023,JESSICA LIDIA DE SOUZA,27/04 Villa - Jessica entrada 11:00 ( À pedi...,NaN,27/04/2023,VILLA,NaN,11:00,13:00,02:00,manha,SEMANA - MANHA - C.CIRURGICO,semana
2,[27/04/2023 11:42:57] Pedro Henrique Diniz Bot...,27/04/2023,PEDRO HENRIQUE DINIZ BOTELHO,27/04 Jabaquara - Diniz entrada 5:50,NaN,27/04/2023,JABAQUARA,NaN,05:30,07:00,01:30,6h,SEMANA - 6HS,semana
3,[27/04/2023 13:21:17] Alexsander Takashi Hamad...,27/04/2023,ALEXSANDER TAKASHI HAMADA,27/04 Analia Alexsander entrada 5:15,NaN,27/04/2023,ANALIA,NaN,05:00,07:00,02:00,6h,SEMANA - 6HS,semana
4,[27/04/2023 13:26:26] José Marcos Vieira da Si...,27/04/2023,JOSE MARCOS VIEIRA DA SILVA,27/04 - Itaim José Marcos entrada 6:00,NaN,27/04/2023,ITAIM,NaN,06:00,07:00,01:00,6h,SEMANA - 6HS,semana


In [10]:
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        print('index', ind)
        print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        print('codigos:', unidade, setorzin)
        
        break

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


In [ ]:
print(df['unidade'][X])
print(df['setor'][X])

# RODANDO NO PEGA

In [11]:
len(df)

129

In [1]:
H=0

In [22]:
for h in range(H, len(df)):
    print(h)
    #click criar novo plantão
    browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
    #esta disponivel o botão de setor
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
    #Setor====
    print('base:', df['unidade'][h], '-', df['setor'][h])
    browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
    ind = scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][h]) & 
                             (scrap_unidades.momento == df['setor'][h])].index.values[0]
    unidade = scrap_unidades['unidade_count'][ind]
    setorzin = scrap_unidades['plt_count'][ind]
    print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
    #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
    print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
          browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))
    
    #listar todos
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()
    
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    sleep(0.5)
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(df['nome'][h])
    #Dando enter
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
    #info
    print('base:', df['nome'][h])
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
    print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)
    
    sleep(2)
    #Data
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(df['data'][h])
    sleep(0.5)
    print(df['data'][h])
    
    #Inicio do plantão
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys(df['ENTRADA'][h])
    sleep(1)
    print('hora de entrada:', df['ENTRADA'][h], '\nhora de saida:', df['SAIDA'][h])
    #Duração do plantão
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys(df['DURAÇÃO'][h])
    print('duração do plantão', df['DURAÇÃO'][h])
        
    ##TIPO
    '''
    //*[@id="select2-drop"]/ul/li[3]/div
    1 - Normal
    2 - DESLOCAMENTO
    3 - VOLUNTÁRIO
    4 - COBERTURA
    '''
    if df['extra'][h] == 'extra':
        print('EXTRA')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[3]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(200) #Pontos
            print('duração >5 -> 200 pontos')
        else:
            pass
    else:
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(2)
        
    #salvar plantão
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
        #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
        sleep(1)
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
        df.loc[int(h), 'controle'] = "PROBS"
        print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"
    
    #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

    #não esta disponivel o X
    
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    print('='*40, '\n')

110
base: OSASCO - SEMANA - 6HS
scraping: OSASCO - SEMANA - 6HS
caixa: Osasco - Semana - 6hs
selecionado: Semana - 6hs 

base: ARTHUR RODRIGUES TORRELIO
selecionado: ARTHUR RODRIGUES TORRÉLIO

28/04/2023
hora de entrada: 06:00 
hora de saida: 07:00
duração do plantão 01:00

111
base: OSASCO - SEMANA - 6HS
scraping: OSASCO - SEMANA - 6HS
caixa: Osasco - Semana - 6hs
selecionado: Semana - 6hs 

base: LUDMYLLA GOMES NEVES
selecionado: LUDMYLLA GOMES NEVES

28/04/2023
hora de entrada: 05:30 
hora de saida: 07:00
duração do plantão 01:30

112
base: VILLA - SEMANA - 6HS
scraping: VILLA - SEMANA - 6HS
caixa: Villa - Semana - 6hs
selecionado: Semana - 6hs 

base: JULIANA HUSSEIN GRILO
selecionado: JULIANA HUSSEIN GRILO

28/04/2023
hora de entrada: 06:00 
hora de saida: 07:00
duração do plantão 01:00

113
base: ANALIA - SEMANA - 6HS
scraping: ANALIA - SEMANA - 6HS
caixa: Anália - Semana - 6hs
selecionado: Semana - 6hs 

base: CARLOS EDUARDO CORREA GRILO
selecionado: CARLOS EDUARDO CORRÊA GRILO


In [37]:
print(df['controle'].value_counts())

NORM     236
PROBS     73
Name: controle, dtype: int64


In [ ]:
df['nome'][h]

In [21]:
df['nome'][df['nome'] == 'ARTHUR TORRELIO'] = 'ARTHUR RODRIGUES TORRELIO'

In [ ]:
print(df['nome'][H])
df.loc[int(H), 'controle'] = "PROBS"
H += 1
print(df['nome'][H])

In [ ]:
print(df['nome'][H])

In [ ]:
len(df)

# Testes:

In [ ]:
df.head()

In [ ]:
browser.find_element("xpath", '/html/body/div[6]/div/div[2]/div[2]/div/label').text

In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').is_enabled()

In [ ]:
df['nome'][df['nome'] == "MICHELE DE OLIVEIRA CHAGAS"] = "MICHELLE DE OLIVEIRA CHAGAS"

In [ ]:
df["controle"].iloc[344]

In [ ]:
browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text

In [ ]:
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[3]/div').click()

In [ ]:
browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()

In [ ]:
browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text

In [ ]:
browser.find_element("xpath", '//*[@id="txtServiceDuration"]').click()

In [ ]:
#browser.find_element_by_xpath('/html/body/div[5]/div/div[2]/div[2]/div/label')

browser.find_element_by_xpath("/html/body/div[4]/div/div[2]/div[2]/div/label").text

# Salvando restante:

In [23]:
df['controle'].value_counts()

NORM     98
PROBS    31
Name: controle, dtype: int64

In [24]:
len(df)

129

In [25]:
# Registros com Problemas
df_giov = pd.read_excel('df_giov.xlsx')
print("giovaneli:", len(df_giov))

# Registros com Problemas Corrigiveis ou Não
df_gabs = df[df['controle'] != 'NORM']
df_gabs.reset_index(drop=True, inplace=True)
print("probs:", len(df_gabs))

# Registros Perfeitos ja feitos
df = df[df['controle'] == 'NORM']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df))

#salvando
df.to_excel("df_gabs.xlsx", index=False)
df_gabs.to_excel("df_gabs2.xlsx", index=False)

giovaneli: 23
probs: 31
ok: 98


In [ ]:
browser.quit()